In [16]:
import pandas as pd
import numpy as np
import datetime
import re

import warnings
warnings.filterwarnings('ignore')

In [17]:
data_channel = pd.read_csv("data/log_channel.csv")
data_playlist = pd.read_csv("data/playlist.csv")

In [18]:
print (data_channel.shape)
print (data_playlist.shape)

(50000, 55)
(81381, 18)


Удаляем полностью пустые

In [19]:
data_playlist = data_playlist.dropna(axis=1,how='all')
data_channel = data_channel.dropna(axis=1,how='all')

Обработка времени

In [20]:
data_channel['event_start'] = pd.to_datetime(data_channel['event_start'])
data_channel['event_end'] = pd.to_datetime(data_channel['event_end'])

data_channel['program_start'] = pd.to_datetime(data_channel['program_start'])
data_channel['program_stop'] = pd.to_datetime(data_channel['program_stop'])

data_channel['event_start'] = pd.to_datetime(data_channel['event_start'])
data_channel['event_stop'] = pd.to_datetime(data_channel['event_end'])

data_channel['dt_add_row'] = pd.to_datetime(data_channel['dt_add_row'])

data_channel['content_aired_start'] = pd.to_datetime(data_channel['content_aired_start'])
data_channel['content_aired_end'] = pd.to_datetime(data_channel['content_aired_end'])

data_playlist['dt_add'] = pd.to_datetime(data_playlist['dt_add'])
data_playlist['dt_event_start'] = pd.to_datetime(data_playlist['dt_event'])

data_playlist['time_added'] = pd.to_timedelta(data_playlist['duration'],'s')

data_playlist['dt_event_end'] = data_playlist['dt_event_start'] + data_playlist['time_added']

In [ ]:
for i in list(data_channel):
    print (i + ": " + str(data_channel[i][0]))

In [ ]:
# IDFA - Identifier for advertisers, Apple's alternative to HTTP cookies on iOS
# AAID provides the same type of device-specific, unique, resettable ID for advertising as the IDFA in Android

In [ ]:
# event_start event_end user_id

In [ ]:
for i in list(data_playlist):
    print (i + ": " + str(data_playlist[i][0]))

In [ ]:
# dt_events
# durations - seconds

In [ ]:
# необходимо расставить метки на логи просмотра где был онлайн весь период показа рекламы

Ищем записи, когда ролик был полностью просмотрен

In [ ]:
user_ids_with_1_labels = []

In [ ]:
num = 0

for index1, row1 in data_playlist.iterrows():
    start_date = row1['dt_event_start']
    end_date = row1['dt_event_end']

    mask = (data_channel['event_start'] <= start_date) & (data_channel['event_end'] >= end_date)
    
    df = data_channel.loc[mask] 
    
    for index2, row2 in df.iterrows():
        user_id = row2['user_id']
        if not(user_id in user_ids_with_1_labels):
            user_ids_with_1_labels.append(user_id)
    
    if num % 1000 == 0:
        print (num)
        
    num += 1

In [ ]:
with open("data/ids", "w") as f:
    for i in user_ids_with_1_labels:
        f.write(i + "\n")

In [21]:
user_ids_with_1_labels = []
with open("data/ids", "r") as f:
    for i in f.readlines():
        s = i.replace("\n", "")
        user_ids_with_1_labels.append(s)

Работа с фичами

In [ ]:
# main prepare data

In [22]:
to_drop = ['id', 'session_id', 'operator_id', 'operator_channel_id', 'user_id',
'event_start', 'event_start_date', 'event_start_hour', 'event_start_minut', 'event_start_second',
'event_end', 'event_end_date', 'event_end_hour', 'event_end_minut', 'event_end_second', 'event_stop',
'program_start', 'program_stop', 'content_aired_end', 'content_aired_start', 'dt_add_row']

categorical = ['operator_name', 'channel_name', 'vi_channel_id', 'app_id',
'device_ip', 'device_type_name', 'geo_continent_code', 'geo_subdivision_1_name', 'geo_country_iso_code',
'geo_subdivision_2_name', 'geo_subdivision_1_iso_code', 'geo_subdivision_2_iso_code',
'geo_city_name', 'geo_time_zone', 'geo_country_name', 'geo_continent_name', 'program_ganre_name', 'program_topic_name',
'program_country_name', 'program_category_name', 'program_title_name', 'user_agent']

numer_not_prepare = ['mode_view', 'device_type', 'event_duration', 'program_rating', 
'program_user_dur_view_prog', 'procent_user_view_prog', 'program_duration', 'region_streaming',
'content_aired_duration', 'stream_type']

numer_need_prepare = ['user_agent', 'dt_add_row']

dt_add_row -> dt_add_row_since_epoch

In [23]:
epoch = datetime.datetime.utcfromtimestamp(0)

def unix_time_millis(dt):
    return (dt - epoch).total_seconds()

data_channel['dt_add_row_since_epoch'] = None
data_channel['dt_add_row_since_epoch'] = data_channel['dt_add_row'].apply(unix_time_millis)

Извлекаем фичи из user_agent

In [24]:
lst = []
for i in data_channel['user_agent']:
    if not("android" in i) and not("apple" in i):
        lst.append(i)
    
with open("data/user_agenst_all", "w") as f:
    for i in list(set(lst)):
        f.write(i + "\n")

In [25]:
data_channel['mobile'] = None
data_channel['android'] = None
data_channel['apple'] = None

data_channel['user_agent'] = data_channel['user_agent'].str.lower()

data_channel['mobile'][data_channel['user_agent'].str.contains("android|apple", na=False)] = 1
data_channel['android'][data_channel['user_agent'].str.contains("android", na=False)] = 1
data_channel['apple'][data_channel['user_agent'].str.contains("android", na=False)] = 1

data_channel['mobile'][data_channel['mobile'] != 1] = 1
data_channel['android'][data_channel['android'] != 1] = 1
data_channel['apple'][data_channel['apple'] != 1] = 1

# можно ещё подаставать фичи

Кодируем категориальные фичи

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
for i in categorical:
    label = LabelEncoder()
    dicts = {}
    
    data_channel[i] = data_channel[i].apply(str)

    label.fit(data_channel[i].drop_duplicates()) #задаем список значений для кодирования
    dicts[i] = list(label.classes_)
    data_channel[i] = label.transform(data_channel[i]) #заменяем значения из списка кодами закодированных элементов 

Метки (0 - ролик не смотрели, 1 - ролик смотрели)

In [28]:
data_channel['label'] = None
data_channel['label'][data_channel['user_id'].isin(user_ids_with_1_labels)] = 1
data_channel['label'][data_channel['label'] != 1] = 0

print (data_channel['label'][data_channel['label'] == 1].shape[0])
print (data_channel['label'][data_channel['label'] != 1].shape[0])

38426
11574


Финальная обработка

In [29]:
data_channel = data_channel.drop(to_drop, 1)

In [30]:
def replace_str(str_s):
    return re.sub("\D", "", str_s)
    
for i in list(data_channel):
    if isinstance(data_channel[i][0] , str):
        data_channel[i] = data_channel[i].apply(replace_str)
        data_channel[i] = data_channel[i].astype(int)

data_channel = data_channel.fillna(0)

In [31]:
data_channel.to_csv('data/prepared_data', header=False, index=False)
data_channel.to_csv('data/prepared_data_pandas', index=False)